|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating layers<h1>|
|<h2>Lecture:</h2>|<h1><b>Mutual information vs. covariance<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_selection import mutual_info_regression

# vector matplotlib
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Example cases

In [ ]:
# parameters
N = 99
eps = 1e-14


### generate data
x = np.random.randn(N)
y1 = x*.5 + np.random.randn(N)*np.sqrt(1-.7**2)
y2 = -y1
y3 = x**2 + np.random.randn(N)/2
y4 = np.cos(np.pi*x) + np.random.randn(N)/3


_,axs = plt.subplots(2,2,figsize=(10,7))
axs = axs.flatten()

for i,y in enumerate((y1,y2,y3,y4)):

  ### compute covariance
  covariance = np.cov(x,y)[0,1]

  ### compute mutual information
  mutinfo = mutual_info_regression(x.reshape(-1,1),y)[0]


  axs[i].plot(x,y,'bo',markersize=10,markerfacecolor=[.7,.7,.9,.5],)
  axs[i].set(xlabel='Data "x"',ylabel='Data "y"',title=f'Covar.: {covariance:.2f}, mutual info: {mutinfo:.2f}')

plt.tight_layout()
plt.show()

# More on the linear situation

In [ ]:
# range of population correlation coefficients
rs = np.linspace(-.9,.9,100)

# sample size
N = 500

# initialize output matrix
measures = np.zeros((len(rs),3))


# loop over a range of r values
for ri in range(len(rs)):

  ### generate data
  x = np.random.randn(N)
  y = x*rs[ri] + np.random.randn(N)*np.sqrt(1-rs[ri]**2)

  ### compute covariance
  measures[ri,0] = np.cov(x,y)[0,1]

  ### compute mutual information
  measures[ri,1] = mutual_info_regression(x.reshape(-1,1),y)[0]

  ###
  Z,xx,yy = np.histogram2d(x,y,bins=15)
  p_Z = Z / Z.sum()
  p_x = np.sum(p_Z, axis=1)
  p_y = np.sum(p_Z, axis=0)

  # calculate entropy
  eps = 1e-13
  entropy_x = -np.sum( p_x * np.log2(p_x+eps) )
  entropy_y = -np.sum( p_y * np.log2(p_y+eps) )
  entropy_Z = -np.sum(p_Z * np.log2(p_Z + eps))
  measures[ri,2] = (entropy_x+entropy_y) - entropy_Z



## visualize the results
_,axs = plt.subplots(2,2,figsize=(10,7))

axs[0,0].plot(rs,measures[:,0],'rs',markersize=8,markerfacecolor=[.9,.7,.7,.5],label='Covariance')
axs[0,0].plot(rs,measures[:,1],'bo',markersize=8,markerfacecolor=[.7,.7,.9,.5],label='Mutual information')
axs[0,0].legend()
axs[0,0].set(xlabel='Population correlation',ylabel=r'Measured $r$ or MI',title='Covariance and mutual information')

axs[0,1].plot(measures[:,0],measures[:,1],'ks',markersize=8,markerfacecolor=[.7,.9,.7,.5])
axs[0,1].axhline(y=0,color='gray',linestyle='--')
axs[0,1].axvline(x=0,color='gray',linestyle='--')
axs[0,1].set(xlabel='Covariance',ylabel='Mutual information',title=f'$r=${np.corrcoef(measures.T)[1,0]:.2f}')


axs[1,0].plot(rs,measures[:,0]**2,'rs',markersize=8,markerfacecolor=[.9,.7,.7,.5],label='(covariance)$^2$')
axs[1,0].plot(rs,measures[:,1],'bo',markersize=8,markerfacecolor=[.7,.7,.9,.5],label='Mutual information')
axs[1,0].legend()
axs[1,0].set(xlabel='Population correlation',ylabel=r'Measured $Cov^2$ or $S_C$',title='Squared covariance and mutual information')

axs[1,1].plot(measures[:,0]**2,measures[:,1],'ks',markersize=8,markerfacecolor=[.7,.9,.7,.5])
axs[1,1].set(xlabel='(covariance)$^2$',ylabel='Mutual information',title=f'$r=${np.corrcoef(abs(measures[:,0]),measures[:,1])[1,0]:.2f}')



plt.tight_layout()
plt.show()

# Comparing manual and Sklearn's implementations

In [ ]:
plt.plot(measures[:,1],measures[:,2],'ko',markerfacecolor=[.7,.7,.7,.5])
plt.plot([0,1],[0,1],linestyle='--',color='k',label='Unity')
plt.legend()
plt.gca().set(xlabel='SKlearn',ylabel='Manual',xlim=[-.02,None],ylim=[-.02,None],
              title=f'r = {np.corrcoef(measures[:,1:].T)[0,1]:.3f}')
plt.show()